# Explaining how to solve a Sudoku

This notebook covers how to generate a sequence of explanations for Constraint Satisfaction problems. Here the use case is: 

You are solving a Sudoku, but at one point, you don't know how to continue. Is there a _HINT_ button I can press to help me out and tell me which number I should write next? 

The answer is YES! 

In this notebook, we present how the reasoning behind this button works in practice. We show how to generate these ___explanations___ and how you can adapt them in order to fit your preferences.

First we load the CPMpy library:

In [19]:
# load the libraries
import sys
sys.path.append("/Users/emiliogamba/Documents/01_VUB/01_Research/01_Shared_Projects/07_CPMPY")

import numpy as np # data library
from cpmpy import * # CP Modeling Library
from IPython.display import display, HTML # display some titles

## 1. Modeling the Sudoku

In practice, some puzzlers like annotate their sudoku with the numbers that cannot be selected. To keep the explanations limited,in this notebook, we model the sudoku using integer variables and consider only explanations consisting of positive assignments. 

In [2]:
def model_sudoku(given):
    # Variables
    puzzle = intvar(1,9, shape=given.shape, name="puzzle")

    model = Model(
        # Constraints on values (cells that are not empty)
        puzzle[given!=e] == given[given!=e], # numpy's indexing, vectorized equality
        # Constraints on rows and columns
        [AllDifferent(row) for row in puzzle],
        [AllDifferent(col) for col in puzzle.T], # numpy's Transpose
    )

    # Constraints on blocks
    for i in range(0,9, 3):
        for j in range(0,9, 3):
            model += AllDifferent(puzzle[i:i+3, j:j+3]) # python's indexing

    return model, puzzle

Pretty printing of a sudoku grid

In [16]:
def print_sudoku(grid):
    out = ""
    for r in range(0,9):
        for c in range(0,9):
            out += str(grid[r, c])
            out += '* ' if grid[r, c] else '  '
            if (c+1) % 3 == 0 and c != 8: # end of block
                out += '| '
        out += '\n'
        if (r+1) % 3 == 0 and r != 8: # end of block
            out += ('-'*9)+'+-'+('-'*9)+'+'+('-'*9)+'\n'
    print(out)   

## Solving the Sudoku

Call the solver on the Sudoku model and extract the solution.

In [17]:
def extract_solution(model, variables, verbose=False):
    ncols = nrows = 9
    solved = model.solve()
    assert solved, "Model is unsatisfiable."
    
    sol = np.array([
        [variables[r, c].value() for c in range(ncols)] for r in range(nrows)
    ])
     
    return sol
    

## Example SUDOKU: Model and Solve

The following grid is an example of a Sudoku grid, where the given values have a value greater than 0 and the others are fixed to 0.

In [28]:
e = 0 # value for empty cells
given = np.array([
    [6, 9, 4,  e, e, 1,  e, e, e],
    [e, e, 3,  e, 2, e,  e, 4, 5],
    [2, 7, e,  e, 6, e,  e, e, e],

    [e, e, 1,  e, e, 4,  e, 7, e],
    [e, 2, 6,  e, 8, 7,  e, 9, 3],
    [3, 5, 7,  e, e, e,  4, e, 2],

    [e, 6, 9,  e, e, e,  e, e, 1],
    [1, 3, e,  6, e, e,  7, e, e],
    [e, e, e,  1, e, 2,  e, e, e]]
)

display(HTML('<h3> ------- INPUT SUDOKU -------</h3>'))
print_sudoku(given)

sudoku_model, sudoku_vars = model_sudoku(given)

sudoku_solution = extract_solution(sudoku_model, sudoku_vars)


display(HTML('<h3>---------- SOLUTION ---------- </h3>'))
print_sudoku(sudoku_solution)

6* 9* 4* | 0  0  1* | 0  0  0  
0  0  3* | 0  2* 0  | 0  4* 5* 
2* 7* 0  | 0  6* 0  | 0  0  0  
---------+----------+---------
0  0  1* | 0  0  4* | 0  7* 0  
0  2* 6* | 0  8* 7* | 0  9* 3* 
3* 5* 7* | 0  0  0  | 4* 0  2* 
---------+----------+---------
0  6* 9* | 0  0  0  | 0  0  1* 
1* 3* 0  | 6* 0  0  | 7* 0  0  
0  0  0  | 1* 0  2* | 0  0  0  



6* 9* 4* | 8* 5* 1* | 2* 3* 7* 
8* 1* 3* | 7* 2* 9* | 6* 4* 5* 
2* 7* 5* | 4* 6* 3* | 9* 1* 8* 
---------+----------+---------
9* 8* 1* | 2* 3* 4* | 5* 7* 6* 
4* 2* 6* | 5* 8* 7* | 1* 9* 3* 
3* 5* 7* | 9* 1* 6* | 4* 8* 2* 
---------+----------+---------
7* 6* 9* | 3* 4* 5* | 8* 2* 1* 
1* 3* 2* | 6* 9* 8* | 7* 5* 4* 
5* 4* 8* | 1* 7* 2* | 3* 6* 9* 



## Explanations for SUDOKU: 

To explain a full sudoku from the givens to the solution, we generate a sequence of intermediate expalnation steps.
Every explanation step is characterized by an *interpretation*, which corresponds to current status of the grid. 
The initial state of the gris is called the ***initial interpretation***, and the solution is also known as the ***end interpretation***.

Every explanation step uses subset of the problem constraints and part of the interpretation in order to derive one or multiple new numbers. 

1. __C__' ⊂ __C__ A subset of the problem constraints (alldifferents on columns, rows and blocks).

2. __I__' ⊂ __I__ A partial interpretation

    - In the Sudoku case this corresponds to the numbers filled in the grid at the current explanation step (givens, and newly derived numbers).

3. __N__ A newly found number to fill in the grid.

Therefore at every step, the number 

To compute such explanations, we take advantage of the link between non-redundant explanations and Minimal Unsatisfiable Subsets introduced in [1]. 


[1] Bogaerts, B., Gamba, E., & Guns, T. (2021). A framework for step-wise explaining how to solve constraint satisfaction problems. Artificial Intelligence, 300, 103550.

### Propagation of constraints 


In [38]:
def solution_intersection(model, solver="ortools", verbose=False):
    """
        solution_intersection produces the intersection of all models
    """
    # Build sat model
    sat_vars = get_variables_model(model)

    SAT = SolverLookup.lookup(solver)(model)

    assert SAT.solve(), "Propagation of soft constraints only possible if model is SAT."
    sat_model = set(bv if bv.value() else ~bv for bv in sat_vars)

    while(SAT.solve()):
        # negate the values of the model
        sat_model &= set(bv if bv.value() else ~bv for bv in sat_vars)
        blocking_clause = ~all(sat_model)

        SAT += blocking_clause

        if verbose:
            print("\n\tBlocking clause:", blocking_clause)

    return sat_model

In [37]:
## Examining the Model constraints$
from cpmpy.transformations.flatten_model import flatten_constraint

falttened_constraints = flatten_constraint(sudoku_model.constraints)

[puzzle[0,0] == 6, puzzle[0,1] == 9, puzzle[0,2] == 4, puzzle[0,5] == 1, puzzle[1,2] == 3, puzzle[1,4] == 2, puzzle[1,7] == 4, puzzle[1,8] == 5, puzzle[2,0] == 2, puzzle[2,1] == 7, puzzle[2,4] == 6, puzzle[3,2] == 1, puzzle[3,5] == 4, puzzle[3,7] == 7, puzzle[4,1] == 2, puzzle[4,2] == 6, puzzle[4,4] == 8, puzzle[4,5] == 7, puzzle[4,7] == 9, puzzle[4,8] == 3, puzzle[5,0] == 3, puzzle[5,1] == 5, puzzle[5,2] == 7, puzzle[5,6] == 4, puzzle[5,8] == 2, puzzle[6,1] == 6, puzzle[6,2] == 9, puzzle[6,8] == 1, puzzle[7,0] == 1, puzzle[7,1] == 3, puzzle[7,3] == 6, puzzle[7,6] == 7, puzzle[8,3] == 1, puzzle[8,5] == 2, alldifferent(puzzle[0,0],puzzle[0,1],puzzle[0,2],puzzle[0,3],puzzle[0,4],puzzle[0,5],puzzle[0,6],puzzle[0,7],puzzle[0,8]), alldifferent(puzzle[1,0],puzzle[1,1],puzzle[1,2],puzzle[1,3],puzzle[1,4],puzzle[1,5],puzzle[1,6],puzzle[1,7],puzzle[1,8]), alldifferent(puzzle[2,0],puzzle[2,1],puzzle[2,2],puzzle[2,3],puzzle[2,4],puzzle[2,5],puzzle[2,6],puzzle[2,7],puzzle[2,8]), alldifferent(puzzl

In [ ]:
for constraint in sudoku_model.constraints:
    print(f"{constraint=}")
    print(f"{flatten_constraint(constraint)=}")


In [ ]:
def explain_sequence(soft, soft_weights=None,  hard=[], solver="ortools", verbose=0):
    '''
        A SAT-based explanation sequence generating algorithm using assumption
        variables and weighted UNSAT core (Optimal Constrained Unsatisfiable Subset)
        extraction. [1, 2]

        [1] Bogaerts, B., Gamba, E., Claes, J., & Guns, T. (2020). Step-wise explanations
        of constraint satisfaction problems. In ECAI 2020-24th European Conference on
        Artificial Intelligence, 29 August-8 September 2020, Santiago de Compostela,
        Spain, August 29-September 8, 2020-Including 10th Conference on Prestigious
        Applications of Artificial Intelligence (PAIS 2020) (Vol. 325, pp. 640-647).
        IOS Press; https://doi. org/10.3233/FAIA200149.

        [2] Bogaerts, B., Gamba, E., & Guns, T. (2021). A framework for step-wise explaining
        how to solve constraint satisfaction problems. Artificial Intelligence, 300, 103550.
    '''
    curr_sol = set()
    # compute all derivable literals
    full_sol = solution_intersection(Model(hard + soft), solver, verbose)

    # prep soft constraint formulation with a literal for each soft constraint
    # (so we can iteratively use an assumption solver on softliterals)
    soft_lit = BoolVar(shape=len(soft), name="ind")
    reified_soft = []
    for i,bv in enumerate(soft_lit):
        reified_soft += [bv.implies(soft[i])]
    # to map indicator variable back to soft_constraints
    indmap = dict((v,i) for (i,v) in enumerate(soft_lit))

    # Cost function returns cost of soft weight if a constraint is used
    # otherwhise returns 1 (i.e. using a literal)
    cost = cost_func(list(soft_lit), soft_weights)

    if verbose > 0:
        print("\nCurrent Model\n\t", curr_sol)
        print("\nFull Model  (holds in all models) \n\t", full_sol)
        print("\nRemaining to explain:\n\t", full_sol-curr_sol)

    
    # we will explain literal by literal
    explanation_sequence = []
    while(curr_sol != full_sol):
        # explain 1 step using ocus
        remaining_to_explain = full_sol - curr_sol
        all_lit = set(soft_lit) | curr_sol

        ocus_expl = explain_one_step_ocus(hard + reified_soft, all_lit, cost, remaining_to_explain, solver, verbose)

        # project on known facts and constraints
        cons_used =  [soft[indmap[con]] for con in set(soft_lit) & ocus_expl]
        facts_used = curr_sol & ocus_expl
        derived = set(~v for v in ocus_expl - set(soft_lit) - facts_used)

        # Add newly derived information
        curr_sol |= derived

        explanation = {
            "constraints": list(cons_used),
            "facts": list(facts_used),
            "derived": list(derived),
            "cost": sum(cost(con) for con in ocus_expl)
        }

        explanation_sequence.append(explanation)

        if verbose > 0:
            print(f"\n Using:\t {explanation['constraints']} /\\ {explanation['facts']} => {explanation['derived']} [cost: {explanation['cost']}]")

    # return the list of original (non-flattened) constraints
    return explanation_sequence

In [ ]:
def explain_one_step_ocus(hard, soft_lit, cost, remaining_sol_to_explain, solver="ortools", verbose=False):
    """
        Optimal Constrained Unsatisfiable Subsets (OCUS) for CSP explanations [1]

        explain_one_step_ocus relies on the implicit hitting set duality between
        MUSes and MCSes for a given formula F [2, 3]:

            - A set S subseteq F is an MCS of F iff it is a minimal hitting set
             of MUSs(F).
            - A set S subseteq F is a MUS of F iff it is a minimal hitting set
             of MCSs(F).

        Builds a MIP model for computing minimal (optimal) hitting sets. Repeatedly
        checks for satisfiability until UNSAT, which means an OCUS is found.
        If SAT, and sat solver computes a model. The complement w.r.t F is
        computed and added as a new set-to-hit.

        MIP MODEL
        ---------

        The constrained optimal hitting set is described by:

            - x_l={0,1} is a boolean decision variable if the literal is inside the
                        hitting set or not.
            - w_l=f(l) is the cost assigned to having the literal in the hitting
                        set.
            - c_lj={0,1} is 1 (0) if the literal l is (not) present in the set-to-hit j.

        Objective:
                min sum(x_l * w_l) over all l in I + (-Iend setminus -I)

        Subject to:
            (1) sum x_l * c_lj >= 1 for all hitting sets j.
                = The hitting set must hit all sets-to-hit.

            (2) sum x_l == 1 for l in (-Iend setminus -I)
                = exactly 1 literal explained at a time

        Args
        ----

            hard (list[cpmpy.Expression]): Hard constraints

            Iend (set): Cautious consequence, the set of literals that hold in
                        all models.

            I (set): partial interpretation (subset of Iend).

        [1] Gamba, E., Bogaerts, B., & Guns, T. (8 2021). Efficiently Explaining CSPs
        with Unsatisfiable Subset Optimization. In Z.-H. Zhou (Red), Proceedings of the
        Thirtieth International Joint Conference on Artificial Intelligence,
        IJCAI-21 (bll 1381–1388). doi:10.24963/ijcai.2021/191.

        [2] Liffiton, M. H., & Sakallah, K. A. (2008). Algorithms for computing minimal
        unsatisfiable subsets of constraints. Journal of Automated Reasoning, 40(1), 1-33.

        [3] Reiter, R. (1987). A theory of diagnosis from first principles.
        Artificial intelligence, 32(1), 57-95.
    """
    ## Unsatisfiable Formula = soft constraints + (~remaining_to_explain)
    neg_remaining_sol_to_explain = set(~var for var in remaining_sol_to_explain)
    F = set(soft_lit) | neg_remaining_sol_to_explain


    ## ----- CONDITIONAL OPTIMISATION MODEL------
    ## -------------- VARIABLES -----------------
    hs_vars = boolvar(shape=len(soft_lit) + len(remaining_sol_to_explain))

    # id of variables that need to be explained
    remaining_hs_vars = hs_vars[[id for id, var in enumerate(F) if var in neg_remaining_sol_to_explain]]
    # mapping between hitting set variables hs_var <-> Iend
    varmap_hs_sat = dict( (hs_vars[id], var) for id, var in enumerate(F))
    varmap_sat_hs = dict( (var, hs_vars[id]) for id, var in enumerate(F))

    ## ----------------- MODEL ------------------
    hs_mip_model = Model(
         # exactly one variable to explain!
        sum(remaining_hs_vars) == 1,
        # Objective: min sum(x_l * w_l) over all l in I + (-Iend \ -I)
        minimize=sum(hs_var * cost(varmap_hs_sat[hs_var]) for hs_var in hs_vars) 
    )

    # instantiate hitting set solver
    hittingset_solver = SolverLookup.lookup(solver)(hs_mip_model)


    ## ----- SAT solver model ----
    SAT = SolverLookup.lookup(solver)(Model(hard))

    while(True):
        hittingset_solver.solve()

        # Get hitting set
        hs = hs_vars[hs_vars.value() == 1]

        # map to vars of formula F
        S = set(varmap_hs_sat[hs_var] for hs_var in hs)

        if verbose > 1:
            print("\n\t hs =", hs, S)

        # SAT check and computation of model
        if not SAT.solve(assumptions=S):
            if verbose > 1:
                print("\n\t ===> OCUS =", S)

            return S

        # satisfying model
        S = set(v if v.value() else ~v for v in F)

        # compute complement of model in formula F
        C =  F - S

        set_to_hit = set(varmap_sat_hs[var] for var in C)

        # Add complement as a new set to hit: sum x[j] * hij >= 1
        hittingset_solver += (sum(set_to_hit) >= 1)

        if verbose > 1:
            print("\t Complement =", C)
            print("\t set-to-hit =", set_to_hit)



In [ ]:
def cost_func(soft, soft_weights):
    '''
        Example cost function with mapping of indicator constraints to
        corresponding given weight.

        Variables not in the indicator map have a unit weight, which
        corresponds to using a unit boolean variable.
    '''

    def cost_lit(cons):
        # return soft weight if constraint is a soft constraint
        if len(set({cons}) & set(soft)) > 0:
            return soft_weights[soft.index(cons)]
        else:
            return 1

    return cost_lit
